Random Forest Regression Model ~ Jyosna

Reference:

https://ploomber.io/blog/nested-cv/

https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

https://medium.com/@maziarizadi/pickle-your-model-in-python-2bbe7dba2bbb

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,root_mean_squared_error

In [22]:
x_train = pd.read_csv("C:\\Users\\jyosn\\Documents\\GitHub\\Predictive-Maintenance\\Datasets\\ForModelDev\\XTrain.csv")
y_train = pd.read_csv("C:\\Users\\jyosn\\Documents\\GitHub\\Predictive-Maintenance\\Datasets\\ForModelDev\\YTrain.csv")
engine = np.array(pd.read_csv("C:\\Users\\jyosn\\Documents\\GitHub\\Predictive-Maintenance\\Datasets\\ForModelDev\\Engine.csv"))

In [28]:
Y_train=np.asarray(y_train).ravel()
Engine=engine.ravel()

In [ ]:
base model

In [29]:
gss_outer = GroupShuffleSplit(n_splits = 10, train_size = 0.7, random_state = 42) #n_splits refers to number of folds required

for i, (train_index, val_index) in enumerate(gss_outer.split(x_train, Y_train, Engine)):
    xtrain = x_train.iloc[train_index, :] # Train Data from training set (70% split)
    ytrain = y_train.iloc[train_index, :]
    Y_train=np.asarray(ytrain).ravel()
    xval = x_train.iloc[val_index, :] # Validation Data from training set (30% split)
    yval = y_train.iloc[val_index, :]
    
    rf = RandomForestRegressor(n_estimators=100,random_state=42,min_samples_leaf=3)
    rf.fit(xtrain, Y_train)
    print(r2_score(rf.predict(xval), yval))

0.6664938496070634
0.39281851031640014
0.6512813872255199
0.634681682135068
0.5551498664247289
0.5668824700406194
0.48390809485095265
0.584106303762581
0.2644713156465103
0.48766456778336253


hyperparameter tuning using nested cross validation

In [30]:
# configure the cross-validation procedure
from numpy import mean, std


# enumerate splits
outer_results = {"r2":[],"rmse":[]}
for i,(train_ix, test_ix) in enumerate(gss_outer.split(x_train, y_train, engine)):
    # split data
    X_train, X_test = x_train.iloc[train_ix, :], x_train.iloc[test_ix, :]
    Y_train, Y_test = y_train.iloc[train_ix], y_train.iloc[test_ix]
    Engine=engine[train_ix]
    Y_train=np.asarray(Y_train).ravel()
    Engine=Engine.ravel()
    # configure the cross-validation procedure
    gss_inner =  GroupShuffleSplit(n_splits = 3, train_size = 0.7, random_state = 42)
    # define the model
    rf = RandomForestRegressor(random_state=42)
    para={"n_estimators":[100,200,300],"max_depth":[6,8,10,12,14],"min_samples_leaf":[4,6,8,10],"criterion":["squared_error"],"ccp_alpha":[0,1,2] }

    # define search
    search = GridSearchCV(rf, param_grid=para, scoring=['r2','neg_root_mean_squared_error'], cv=gss_inner, refit='r2')
    # execute search
    result = search.fit(X_train, Y_train,groups=Engine)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    r2 = r2_score(Y_test, yhat)
    rmse=root_mean_squared_error(Y_test,yhat)
    # store the result
    outer_results["r2"].append(r2)
    outer_results["rmse"].append(rmse)
    # report progress
    print('>r2=%.3f, est=%.3f, cfg=%s' % (r2, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('R2: %.3f (%.3f)' % (mean(outer_results["r2"]), std(outer_results["r2"])))
print('RMSE: %.3f (%.3f)' % (mean(outer_results["rmse"]), std(outer_results["rmse"])))



>r2=0.687, est=0.575, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 8, 'min_samples_leaf': 4, 'n_estimators': 300}
>r2=0.618, est=0.698, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 8, 'n_estimators': 100}
>r2=0.738, est=0.610, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 200}
>r2=0.724, est=0.533, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 8, 'min_samples_leaf': 4, 'n_estimators': 200}
>r2=0.686, est=0.624, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 300}
>r2=0.624, est=0.637, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 10, 'n_estimators': 300}
>r2=0.695, est=0.595, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 10, 'n_estimators': 200}
>r2=0.653, est=0.664, cfg={'ccp_alpha': 0, 'criterion': 'squared_er

output

>r2=0.687, est=0.575, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 8, 'min_samples_leaf': 4, 'n_estimators': 300}

>r2=0.618, est=0.698, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 8, 'n_estimators': 100}

>r2=0.738, est=0.610, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 200}

>r2=0.724, est=0.533, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 8, 'min_samples_leaf': 4, 'n_estimators': 200}

>r2=0.686, est=0.624, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 300}

>r2=0.624, est=0.637, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 10, 'n_estimators': 300}

>r2=0.695, est=0.595, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 10, 'n_estimators': 200}

>r2=0.653, est=0.664, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 100}

>r2=0.605, est=0.768, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 4, 'n_estimators': 200}

>r2=0.628, est=0.739, cfg={'ccp_alpha': 0, 'criterion': 'squared_error', 'max_depth': 6, 'min_samples_leaf': 10, 'n_estimators': 200}

R2: 0.666 (0.044)

RMSE: 39.489 (3.595)

In [31]:
search = GridSearchCV(rf, param_grid=para, scoring=['r2','neg_root_mean_squared_error'], cv=gss_inner, refit='r2')
result = search.fit(X_train, Y_train,groups=Engine)
# get the best performing model fit on the whole training set
best_model = result.best_estimator_
# evaluate model on the hold out dataset
yhat = best_model.predict(X_test)
# evaluate the model
r2 = r2_score(Y_test, yhat)
rmse=root_mean_squared_error(Y_test,yhat)
# store the result

In [33]:
print(r2,rmse)

0.6277776059115391 41.84085429841219


In [32]:
import pickle
pickle.dump(best_model, open('random_forest_model.pkl','wb'))